In [ ]:
import os
from SpikeTrainUtils import SpikeTrainStats, SpikeTrainPlotter
from IPython.display import HTML, display
import matplotlib.pyplot as plt 
import numpy as np
from matplotlib import cm

In [ ]:
d = './results/'
dirs = [os.path.join(d, o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

sorted_dirs = []
for directory in dirs:
    split_d = directory.split(' ')
    sorted_dirs.append([split_d[1], split_d[3], directory])

sorted_dirs = sorted(sorted_dirs, key = lambda x: (int(x[0]), int(x[1])))

In [ ]:
sts = SpikeTrainStats()

results = []
for directory in sorted_dirs:

    job_dir = os.listdir(directory[2])[-1]
    sub_directory = os.path.join(directory[2], job_dir)

    files = [os.path.join(sub_directory, o) for o in os.listdir(sub_directory) if os.path.isfile(os.path.join(sub_directory,o))]
    config_res = []
    for f in files:
        fname,ext = os.path.splitext(f)
        if ext != '.csv':
            continue
        fr, cvisi, min_time, max_time = sts.calculate(f)
        config_res.append({'population': f.split(' ')[10], 'firing_rate':fr, 'cv_isi':cvisi, 'min_time': min_time, 'max_time':max_time})
    results.append([directory,config_res])
    print results[-1]

In [ ]:
pop_types = ['exc', 'inh', 'ext']
pop_labels = {'exc':'Excitatory', 'inh':'Inhibitory', 'ext':'Injection'}

population_data = {}
for pop_type in pop_types:
    population_data[pop_type] = []

for pop_type in pop_types:
    slim_data = []
    for row in results:     
        for i in range(len(row[1])):
            if row[1][i]['population'] != pop_type:
                continue
            slim_data.append([row[0][0], row[0][1], row[1][i]['firing_rate'], row[1][i]['cv_isi'], row[1][i]['min_time'], row[1][i]['max_time']])
    print (pop_labels[pop_type] + " Population")
    table_head = '<table>'
    table_header = '<tr><th>G Exc</th><th>G Inh</th><th>Firing Rate</th><th>CV ISI</th><th>Min Time</th><th>Max Time</th></tr>'
    table_contents = '<tr>{}</tr>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in slim_data))
    table_tail = '</table>'
    html = table_head + table_header + table_contents + table_tail
    display(HTML(html))
    
    
    pop_array = population_data[pop_type]
    pop_array.append(slim_data)
    population_data[pop_type] = pop_array

In [ ]:
population_data['exc']

In [ ]:
%matplotlib inline

def plot_poulation_heatmap(pop_data, x_ind, y_ind, data_ind, cmap=cm.RdBu, title=None, savefig=False):
    

    data = np.array(pop_data[0])
    x = data[:,x_ind].astype(int)
    y = data[:, y_ind].astype(int)
    z = data[:, data_ind].astype(float)
    
    x=np.unique(x)
    y=np.unique(y)
    X,Y = np.meshgrid(x,y)
    Z = z.reshape(len(y), len(x))
    
    plt.pcolormesh(X,Y,Z, cmap=cmap)
    plt.colorbar()
    if title != None:
        plt.title(title)
    plt.xlabel('GExc')
    plt.ylabel('GInh')
    plt.show()
    
    if saveFig:
        plt.savefig(title + '.png',)


In [ ]:
plot_poulation_heatmap(population_data['exc'], 0, 1, 3, title="CV ISI", savefig=True)

In [ ]:
plot_poulation_heatmap(population_data['exc'], 0, 1, 2, cmap= cm.Spectral, title="Firing Rate", savefig=True)

In [ ]:
plot_poulation_heatmap(population_data['exc'], 0, 1, 3, cmap= cm.PuOr, title="Last Spike Time", savefig=True)